In [1]:
import collections
import pprint

In [2]:
channels = [
    "Pz",
    "C.r",
    "C.b",
    "C.g",
    "light1.specular.G",
    "light1.specular.B",
    "light1.specular.R",
    "B",
    "G",
    "R",
    "A",
    "light1.B",
    "light1.R",
    "light1.g",
]

In [3]:
layers = [channel.rsplit(".", 1) for channel in channels]
layers = [layer if len(layer) > 1 else ["", layer[0]] for layer in layers]
layers = [(layer, component, channel) for (layer, component), channel in zip(layers, channels)]
layers

[('', 'Pz', 'Pz'),
 ('C', 'r', 'C.r'),
 ('C', 'b', 'C.b'),
 ('C', 'g', 'C.g'),
 ('light1.specular', 'G', 'light1.specular.G'),
 ('light1.specular', 'B', 'light1.specular.B'),
 ('light1.specular', 'R', 'light1.specular.R'),
 ('', 'B', 'B'),
 ('', 'G', 'G'),
 ('', 'R', 'R'),
 ('', 'A', 'A'),
 ('light1', 'B', 'light1.B'),
 ('light1', 'R', 'light1.R'),
 ('light1', 'g', 'light1.g')]

In [4]:
groups = collections.defaultdict(list)
for layer, component, channel in layers:
    groups[layer].append((component, channel))
groups

defaultdict(list,
            {'': [('Pz', 'Pz'),
              ('B', 'B'),
              ('G', 'G'),
              ('R', 'R'),
              ('A', 'A')],
             'C': [('r', 'C.r'), ('b', 'C.b'), ('g', 'C.g')],
             'light1.specular': [('G', 'light1.specular.G'),
              ('B', 'light1.specular.B'),
              ('R', 'light1.specular.R')],
             'light1': [('B', 'light1.B'),
              ('R', 'light1.R'),
              ('g', 'light1.g')]})

In [5]:
layers = list(groups.items())
layers

[('', [('Pz', 'Pz'), ('B', 'B'), ('G', 'G'), ('R', 'R'), ('A', 'A')]),
 ('C', [('r', 'C.r'), ('b', 'C.b'), ('g', 'C.g')]),
 ('light1.specular',
  [('G', 'light1.specular.G'),
   ('B', 'light1.specular.B'),
   ('R', 'light1.specular.R')]),
 ('light1', [('B', 'light1.B'), ('R', 'light1.R'), ('g', 'light1.g')])]

In [6]:
def split_layers(layers):
    for layer, channels in layers:
        if layer:
            yield (layer, channels)
            continue
        ci_components = [component.lower() for component, channel in channels]
        if "r" in ci_components and "g" in ci_components and "b" in ci_components:
            yield (layer, [channels[ci_components.index("r")], channels[ci_components.index("g")], channels[ci_components.index("b")]])
        channels = [channel for channel, ci_component in zip(channels, ci_components) if ci_component not in "rgb"]
        for channel in channels:
            yield layer, [channel]

In [7]:
layers = list(split_layers(layers))
for layer, channels in layers:
    print(f"{layer:20}{channels}")

                    [('R', 'R'), ('G', 'G'), ('B', 'B')]
                    [('Pz', 'Pz')]
                    [('A', 'A')]
C                   [('r', 'C.r'), ('b', 'C.b'), ('g', 'C.g')]
light1.specular     [('G', 'light1.specular.G'), ('B', 'light1.specular.B'), ('R', 'light1.specular.R')]
light1              [('B', 'light1.B'), ('R', 'light1.R'), ('g', 'light1.g')]


In [8]:
def organize_layers(layers):
    for layer, channels in layers:
        ci_components = [component.lower() for component, channel in channels]
        for collection in ["rgb", "hsv", "hsb", "xy", "xyz", "uv", "uvw"]:
            if sorted(ci_components) == sorted(collection):
                channels = [channels[ci_components.index(component)] for component in collection]            
        yield layer, channels

In [9]:
layers = list(organize_layers(layers))
for layer, channels in layers:
    print(f"{layer:20}{channels}")

                    [('R', 'R'), ('G', 'G'), ('B', 'B')]
                    [('Pz', 'Pz')]
                    [('A', 'A')]
C                   [('r', 'C.r'), ('g', 'C.g'), ('b', 'C.b')]
light1.specular     [('R', 'light1.specular.R'), ('G', 'light1.specular.G'), ('B', 'light1.specular.B')]
light1              [('R', 'light1.R'), ('g', 'light1.g'), ('B', 'light1.B')]


In [10]:
def categorize_layers(layers):
    for layer, channels in layers:
        ci_components = [component.lower() for component, channel in channels]
        print(ci_components)
        if ci_components == ["r", "g", "b"]:
            yield layer, channels, "rgb"
        else:
            yield layer, channels, ""

In [11]:
layers = list(categorize_layers(layers))

['r', 'g', 'b']
['pz']
['a']
['r', 'g', 'b']
['r', 'g', 'b']
['r', 'g', 'b']


In [14]:
for layer, channels, category in layers:
    print(f"{layer:20}{channels!r:85}{category}")

                    [('R', 'R'), ('G', 'G'), ('B', 'B')]                                                 rgb
                    [('Pz', 'Pz')]                                                                       
                    [('A', 'A')]                                                                         
C                   [('r', 'C.r'), ('g', 'C.g'), ('b', 'C.b')]                                           rgb
light1.specular     [('R', 'light1.specular.R'), ('G', 'light1.specular.G'), ('B', 'light1.specular.B')] rgb
light1              [('R', 'light1.R'), ('g', 'light1.g'), ('B', 'light1.B')]                            rgb
